### **Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [57]:
import pandas as pd
keywords = pd.read_csv('keywords.csv')
keywords.head()

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def geo_tag (row):
    
    for key, value in geo_data.items():
        for item in value:
            if item in row['keyword']:
                label = key
                return label

    return 'undefined'
        
keywords['region'] = keywords.apply (geo_tag, axis = 1)

#keywords[keywords['keyword'].str.contains('сахалин', case=False) ].head()
#keywords.head()


,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


### **Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [58]:
import pandas as pd
ratings = pd.read_csv('ratings.csv')
ratings.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
movies = pd.read_csv('movies.csv')
movies.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
joined = movies.merge (ratings, on='movieId', how='left')
grouped = joined.groupby('title').mean().reset_index()[['title','rating']]

def movie_class (row):
    if row['rating'] <= 2:
        return 'low rating'
    elif row['rating'] > 2 and row['rating'] <= 4:
        return 'average rating'
    else:
        return 'high rating'
    
grouped['class'] = grouped.apply(movie_class, axis = 1)
grouped.head()

,title,rating,class
0,'71 (2014),4.0,average rating
1,'Hellboy': The Seeds of Creation (2004),4.0,average rating
2,'Round Midnight (1986),3.5,average rating
3,'Salem's Lot (2004),5.0,high rating
4,'Til There Was You (1997),5.0,high rating


In [36]:
len(movies) == len(joined)

True

### **Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.

In [68]:
ratings = pd.read_csv('ratings.csv')
grouped = ratings.groupby('userId').agg({'timestamp': ['count', 'min', 'max']}).reset_index()
ratings_filtered = grouped[grouped['timestamp']['count'] >= 100]

ratings_filtered ['lifetime'] = ratings_filtered.apply(lambda x: 
                                                        pd.to_datetime(x['timestamp']['max'], unit ='s') - 
                                                             pd.to_datetime(x['timestamp']['min'], unit ='s'), axis = 1)
#ratings_filtered.head()
ratings_filtered['lifetime'].mean().days

C:\Users\Kate\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


511

### **Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга

In [63]:
years = [x for x in range (1950, 2011)]
movies = pd.read_csv('movies.csv')
movies.head()

def production_year (row):
    for i in range(len(years)):
        if str(years[i]) in row['title']:
            return years[i]
    return 1900

movies['year'] = movies.apply (production_year, axis = 1)

ratings = pd.read_csv('ratings.csv')
ratings.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)
movies.drop_duplicates(subset = 'movieId', keep = 'last', inplace = True)

joined = movies.merge (ratings, on='movieId', how='left')
grouped = joined.groupby('year').mean().reset_index()[['year','rating']]
grouped.sort_values('rating', ascending=False)

,year,rating
23,1972,3.800000
9,1958,3.774194
8,1957,3.727273
12,1961,3.696970
20,1969,3.694444
21,1970,3.687500
27,1976,3.670455
6,1955,3.666667
3,1952,3.656250
5,1954,3.652174


In [59]:
len(movies) == len(joined)

True

Верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже?
Ответ - верно.